In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

driver = webdriver.Chrome()

driver.get('https://stockanalysis.com/stocks/')

wait = WebDriverWait(driver, 20)

symbol_list = []
company_name_list = []

def parse_table():
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('div', {'class': 'table-wrap svelte-172ru7t'}).find('table')
    if table:
        rows = table.find_all('tr')[1:] 
        for row in rows:
            cells = row.find_all('td')
            if len(cells) >= 2: 
                symbol = cells[0].text.strip()
                company_name = cells[1].text.strip()
                symbol_list.append(symbol)
                company_name_list.append(company_name)

current_page = 1
total_pages = 12

while current_page <= total_pages:
    parse_table()  
    try:
        next_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Next"]')))
        next_button.click()
        time.sleep(3)  
        current_page += 1
    except Exception as e:
        print(f"Failed to load page {current_page}: {e}")
        break

driver.quit()

if symbol_list and company_name_list:
    df = pd.DataFrame({
        'Symbol': symbol_list,
        'Company Name': company_name_list
    })
    df.to_excel('all_stocks_data.xlsx', index=False)
    print("Data saved to all_stocks_data.xlsx")
else:
    print("No data extracted")

Failed to load page 12: Message: 

Data saved to all_stocks_data.xlsx


In [9]:
df

,Symbol,Company Name
0,A,"Agilent Technologies, Inc."
1,AA,Alcoa Corporation
2,AACG,ATA Creativity Global
3,AACI,Armada Acquisition Corp. I
4,AACT,Ares Acquisition Corporation II
...,...,...
5564,ZVRA,"Zevra Therapeutics, Inc."
5565,ZVSA,"ZyVersa Therapeutics, Inc."
5566,ZWS,Zurn Elkay Water Solutions Corporation
5567,ZYME,Zymeworks Inc.


In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import concurrent.futures

class StockDataBot:
    def __init__(self, base_url, symbols, company_names):
        self.base_url = base_url
        self.symbols = symbols
        self.company_names = company_names
        self.data = []

    def fetch_data(self, symbol, company_name):
        url = self.base_url.format(symbol=symbol)
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        table_rows = soup.find_all('tr', class_='border-b border-gray-200 last:border-0 dark:border-dark-700')
        stock_data = {'Company Name': company_name, 'Ticker Symbol': symbol}
        for row in table_rows:
            cells = row.find_all('td')
            if len(cells) == 2:
                key = cells[0].get_text(strip=True)
                value = cells[1].get_text(strip=True)
                stock_data[key] = value
        self.data.append(stock_data)

    def display_data(self):
        df = pd.DataFrame(self.data)
        cols = ['Company Name', 'Ticker Symbol'] + [col for col in df.columns if col not in ['Company Name', 'Ticker Symbol']]
        df = df[cols]
        return df

    def run(self):
        self.data.clear()  
        with concurrent.futures.ThreadPoolExecutor() as executor:
            executor.map(self.fetch_data, self.symbols, self.company_names)
        return self.display_data()

base_url_detail = 'https://stockanalysis.com/stocks/{symbol}/company/'  

data_bot = StockDataBot(base_url_detail, symbol_list, company_name_list)
df2 = data_bot.run()

df2.to_excel('stock_data.xlsx', index=False) 

print("Data saved to 'stock_data.xlsx'")
df2

Data saved to 'stock_data.xlsx'


,Company Name,Ticker Symbol,Country,Founded,Industry,Sector,Employees,CEO,Phone,Website,Exchange,Fiscal Year,Reporting Currency,CIK Code,CUSIP Number,ISIN Number,Employer ID,SIC Code,IPO Date,IPO Price
0,Atlantic American Corporation,AAME,United States,1968,Insurance - Life,Financials,155,Hilton Hatchett Howell Jr.,(404) 266-5500,atlam.com,NASDAQ,January - December,USD,0000008177,048209100,US0482091008,58-1027114,6311,NaN,NaN
1,American Airlines Group Inc.,AAL,United States,1930,Airlines,Industrials,"132,100",Robert D. Isom Jr.,(682) 278-9000,aa.com,NASDAQ,January - December,USD,0000006201,02376R102,US02376R1023,75-1825172,4512,NaN,NaN
2,ATA Creativity Global,AACG,China,1999,Education & Training Services,Consumer Staples,581,Xiaofeng Ma,8610-6518-1133,ir.atai.net.cn,NASDAQ,January - December,CNY,0001420529,00211V106,US00211V1061,NaN,8200,NaN,NaN
3,Armada Acquisition Corp. I,AACI,United States,2020,Shell Companies,Financials,2,Stephen P. Herbert,(215) 543-6886,armadaacq.com,NASDAQ,October - September,USD,0001844817,04208V103,US04208V1035,85-3810850,6770,"Aug 13, 2021",$10.00
4,Altisource Asset Management Corporation,AAMC,United States,2012,Mortgage Finance,Financials,13,William Charles Erbey,(340) 692-1055,altisourceamc.com,NYSEAMERICAN,January - December,USD,0001555074,02153X108,VI02153X1080,66-0783125,6500,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5564,"Zevra Therapeutics, Inc.",ZVRA,United States,2006,Biotechnology,Healthcare,69,Neil F. McFarlane,(321) 939-3416,zevra.com,NASDAQ,January - December,USD,0001434647,NaN,US4884452065,20-5894398,2834,"Apr 16, 2015",NaN
5565,Zymeworks Inc.,ZYME,Canada,2003,Biotechnology,Healthcare,278,Kenneth H. Galbraith C.A.,(604) 678-1388,zymeworks.com,NASDAQ,January - December,USD,0001403752,98985W102,US98985Y1082,NaN,2834,"Apr 28, 2017",NaN
5566,Zurn Elkay Water Solutions Corporation,ZWS,United States,NaN,Pollution & Treatment Controls,Industrials,"2,400",Todd A. Adams,414-643-3000,zurnwatersolutions.com,NYSE,January - December,USD,0001439288,98983L108,US98983L1089,20-5197013,3560,"Mar 29, 2012",NaN
5567,"ZyVersa Therapeutics, Inc.",ZVSA,United States,2014,Biotechnology,Healthcare,7,Stephen C. Glover,908-370-5102,zyversa.com,NASDAQ,January - December,USD,0001859007,NaN,US98987D2018,86-2685744,2834,NaN,NaN
